In [1]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.utils.data import DataLoader
from wordcloud import WordCloud

import numpy as np
import seaborn as sns

from cities.utils.data_loader import ZoningDataset

smoke_test = "CI" in os.environ
n = 20 if smoke_test else 2000
num_samples = 10 if smoke_test else 1000
n_steps = 10 if smoke_test else 2000

from cities.utils.data_grabber import find_repo_root

root = find_repo_root()

In [2]:

residential = pd.read_csv(os.path.join(root, "data/minneapolis/sourced/residential_permits.csv"))

residential = residential.rename(columns={"parcel_id": "parcel",
                                           "YEAR": "year",
                                           "UNITS": "housing_units",
                                           "ACREAGE": "acreage",})

residential["parcel"] = pd.to_numeric(residential["parcel"], errors='coerce')
residential = residential.dropna(subset=["parcel"])
residential["parcel"] = residential["parcel"].astype(int)


residential["year"] = pd.to_numeric(residential["year"], errors='coerce')
residential['year'] = residential['year'].astype(int)




#TODO anything else to drop?
columns_to_drop = ['SDE_ID', 'CO_CODE', 'CTU_CODE', 'CTU_ID', 'COCTU_ID', 'CTU_NAME']
residential = residential.drop(columns=columns_to_drop)

print(residential.columns)


display(residential.head())

print(
residential.shape
)


residential['housing_units'] = residential['housing_units'].astype(int)





residential = residential[residential["year"] <= 2020]


Index(['parcel', 'year', 'TENURE', 'HOUSING_TY', 'HOUSING__1', 'RES_PERMIT',
       'ADDRESS', 'ZIP_CODE', 'ZIP_PLUS_4', 'NAME', 'BUILDINGS',
       'housing_units', 'AGE_RESTRI', 'MEMORY_CAR', 'ASSISTED', 'COM_OFF_RE',
       'acreage', 'SQF', 'PUBLIC_FUN', 'PERMIT_VAL', 'COMMUNITY_'],
      dtype='object')


,parcel,year,TENURE,HOUSING_TY,HOUSING__1,RES_PERMIT,ADDRESS,ZIP_CODE,ZIP_PLUS_4,NAME,...,housing_units,AGE_RESTRI,MEMORY_CAR,ASSISTED,COM_OFF_RE,acreage,SQF,PUBLIC_FUN,PERMIT_VAL,COMMUNITY_
0,2302924240095,2016,RNT,MF5,Multifamily (5 units or more),NU,10 2nd St SE,NaN,NaN,NaN,...,72.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,14158749.0,Urban Center
1,2302924320836,2016,OWN,MF5,Multifamily (5 units or more),TF,100 3rd Ave S,NaN,NaN,The Carlyle,...,1.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,250000.0,Urban Center
2,2202924410102,2016,RNT,MF5,Multifamily (5 units or more),NU,100 Hennepin Ave,NaN,NaN,NaN,...,156.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,31925920.0,Urban Center
3,3002923230134,2015,OWN,DTQ,"Duplex, triplex and quad",RM,1000 Essex St SE,NaN,NaN,NaN,...,4.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,351000.0,Urban Center
4,2402924310002,2010,RNT,MF5,Multifamily (5 units or more),NU,1000 University Ave SE,55414.0,NaN,FloCo Fusion,...,84.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,10811000.0,Urban Center


(4215, 21)


In [3]:


residential['census_tract'] = np.nan
display(residential.head())

years = sorted(residential['year'].unique().astype(int))
print(years)

,parcel,year,TENURE,HOUSING_TY,HOUSING__1,RES_PERMIT,ADDRESS,ZIP_CODE,ZIP_PLUS_4,NAME,...,AGE_RESTRI,MEMORY_CAR,ASSISTED,COM_OFF_RE,acreage,SQF,PUBLIC_FUN,PERMIT_VAL,COMMUNITY_,census_tract
0,2302924240095,2016,RNT,MF5,Multifamily (5 units or more),NU,10 2nd St SE,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,0.0,0.0,NaN,14158749.0,Urban Center,NaN
1,2302924320836,2016,OWN,MF5,Multifamily (5 units or more),TF,100 3rd Ave S,NaN,NaN,The Carlyle,...,0.0,0.0,0.0,NaN,0.0,0.0,NaN,250000.0,Urban Center,NaN
2,2202924410102,2016,RNT,MF5,Multifamily (5 units or more),NU,100 Hennepin Ave,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,0.0,0.0,NaN,31925920.0,Urban Center,NaN
3,3002923230134,2015,OWN,DTQ,"Duplex, triplex and quad",RM,1000 Essex St SE,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,0.0,0.0,NaN,351000.0,Urban Center,NaN
4,2402924310002,2010,RNT,MF5,Multifamily (5 units or more),NU,1000 University Ave SE,55414.0,NaN,FloCo Fusion,...,0.0,0.0,0.0,NaN,0.0,0.0,NaN,10811000.0,Urban Center,NaN


[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]


In [4]:
for year in years:

    print(year)

    mapping_path = os.path.join(root,
        f"data/minneapolis/sourced/parcel_to_census_tract_mappings/parcel_to_census_tract_ids_{year}.csv")


    mapping_df = pd.read_csv(mapping_path)

    display(mapping_df.head())

    assert mapping_df['parcel_id'].dtype == residential['parcel'].dtype

    mapping_set = set(mapping_df['parcel_id'])
    residential_set = set(residential['parcel'])

    overlap = mapping_set.intersection(residential_set)
    overlap_ratio = len(overlap) / len(residential_set)
    print(f"overlap ratio for year {year}", overlap_ratio)

    residential_year = residential[residential['year'] == year]

    duplicate_parcels = mapping_df[mapping_df.duplicated('parcel_id', keep=False)]
    if len(duplicate_parcels)>0:
        print("duplicate_parels! will keep first!", duplicate_parcels)
        mapping_df = mapping_df.drop_duplicates(subset='parcel_id', keep='first')

    merged = residential_year.merge(mapping_df, how='left', left_on='parcel', right_on='parcel_id')

    merged.set_index(residential_year.index, inplace=True)

    merged['census_tract_id'] = pd.to_numeric(merged['census_tract_id'], errors='coerce')
    merged['parcel_id'] = pd.to_numeric(merged['parcel_id'], errors='coerce')

    merged['census_tract_id'] = merged['census_tract_id'].fillna(0).astype(int)
    merged['parcel_id'] = merged['parcel_id'].fillna(0).astype(int)


    residential.loc[residential['year'] == year, 'census_tract'] = merged['census_tract_id']

residential['census_tract'] = residential['census_tract'].astype(int)
residential.loc[residential['census_tract'] == 0,'census_tract'] = np.nan

print("ratio of parcels without identified tract", residential['census_tract'].isna().sum()/len(residential))


2009


,Unnamed: 0,parcel_id,census_tract_id
0,0,1311821120005,27053000102
1,1,702823320058,27053111100
2,2,702823320056,27053111100
3,3,802824110095,27053109100
4,4,802824110096,27053109100


overlap ratio for year 2009 0.9783464566929134
duplicate_parels! will keep first!         Unnamed: 0      parcel_id  census_tract_id
192            192  2602924320152      27053105400
193            193  2602924320153      27053105400
194            194  2602924320154      27053105400
204            204  3502924330189      27053126000
205            205  3502924330190      27053126000
233            233  1002824210201      27053109900
234            234  1002824210201      27053109900
116858      116858  2602924320152      27053105400
116859      116859  2602924320153      27053105400
116860      116860  2602924320154      27053105400
127814      127814  3502924330189      27053126000
127815      127815  3502924330190      27053126000
2010


,Unnamed: 0,parcel_id,census_tract_id
0,0,1311821120005,27053000102
1,1,702823320058,27053111100
2,2,702823320056,27053111100
3,3,2102924430108,27053104100
4,4,2102924430036,27053104100


overlap ratio for year 2010 0.978740157480315
2011


...

overlap ratio for year 2018 0.8901574803149607
duplicate_parels! will keep first!         Unnamed: 0      parcel_id  census_tract_id
11              11  3602924440191      27053107400
12              12  3602924440191      27053107400
15              15  3602924440187      27053107400
16              16  3602924440187      27053107400
17              17  3602924440187      27053107400
...            ...            ...              ...
138713      138713   102824110028      27053108800
138714      138714   102824110028      27053108800
138715      138715   102824110028      27053108800
138733      138733   102824110007      27053108800
138734      138734   102824110007      27053108800

[19784 rows x 3 columns]
2019


,Unnamed: 0,parcel_id,census_tract_id
0,0,3602924440227,27053107400
1,1,3602924440226,27053107400
2,2,3602924440225,27053107400
3,3,3602924440224,27053107400
4,4,3602924440223,27053107400


overlap ratio for year 2019 0.9047244094488189
2020


,Unnamed: 0,parcel_id,census_tract_id
0,0,3602924440227,27053107400
1,1,3602924440226,27053107400
2,2,3602924440225,27053107400
3,3,3602924440224,27053107400
4,4,3602924440223,27053107400


overlap ratio for year 2020 0.8948818897637796
ratio of parcels without identified tract 0.015029522275899088


In [5]:
before = len(residential)
residential = residential.dropna(subset = ['census_tract'])
after = len(residential)
residential['census_tract'] = residential['census_tract'].astype(int)
print(f"dropped {before - after} from {before} rows as missing census tract")

display(residential['census_tract'].dtype)

dropped 56 from 3726 rows as missing census tract


dtype('int64')